In [86]:
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

# try:
#     # waits up to 10 seconds for the element to be present, checks every 500 ms
#     next_button = WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((By.ID, "ember234"))
#     )
#     next_button.click()

# except Exception as e:
#     print(str(e))



In [87]:

# # If finding by ID fails, try to find the element by aria-label
# next_button = WebDriverWait(driver, 10).until(
#     EC.presence_of_element_located((By.CSS_SELECTOR, "button[aria-label='Next']"))
# )
# next_button.click()



In [88]:
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

# def click_next_button(driver):

#     try:
#         # If finding by ID fails, try to find the element by aria-label
#         next_button = WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.CSS_SELECTOR, "button[aria-label='Next']"))
#         )
#         driver.execute_script("arguments[0].scrollIntoView();", next_button)
#         next_button.click()

#     except Exception as e2:
#         print(f"Failed to find by aria-label, error: {str(e2)}")
# click_next_button(driver)


In [89]:
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC


# try:
#     # Try to find the element by ID
#     next_button = WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((By.ID, "ember234"))
#     )
#     next_button.click()

# except Exception as e:
#     print(f"Failed to find by ID, error: {str(e)}")
#     print("Trying to find by aria-label...")

#     try:
#         # If finding by ID fails, try to find the element by aria-label
#         next_button = WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.CSS_SELECTOR, "button[aria-label='Next']"))
#         )
#         next_button.click()
        
#     except Exception as e2:
#         print(f"Failed to find by aria-label, error: {str(e2)}")




In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
from dotenv import load_dotenv
import os
import openai

from bs4 import BeautifulSoup
import re
import time
import pickle


In [2]:
load_dotenv()  # take environment variables from .env.
os.environ["OPENAI_API_KEY"]  = os.getenv("OPENAI_API_KEY")
openai.api_key = os.environ["OPENAI_API_KEY"]

In [3]:


def find_linkedin_info(driver):
#     time.sleep(5)  # Allow the page to load

    # Get the source HTML of the page
    source = driver.page_source

    # Parse the source HTML with BeautifulSoup
    soup = BeautifulSoup(source, 'lxml')

    # Get the text from the parsed HTML
    page_text = soup.get_text()
    
    # Split the text into lines and remove empty lines
    lines = page_text.split('\n')
    non_empty_lines = [line for line in lines if line.strip() != ""]

    # Join the non-empty lines back into a single string
    url_text = "\n".join(non_empty_lines)

    return url_text


In [4]:

import re

class Extractor:
    '''
    
To request an appropriate pattern or string match for this class, you could ask:

"Please provide a string or a regular expression pattern that we should use for the start 
rule or end rule. If you provide a regular expression pattern, please specify that it is 
a regex. Also, note that for regular expressions, we're using Python's 're' module, so the 
pattern should be compatible with it. If you want to extract from the start or end of the 
text when no matching rule is found, please indicate that as well."
    '''

    def __init__(self):
        self.start_rules = []
        self.end_rules = []

    def add_start_rule(self, rule, is_regex=False):
        self.start_rules.append((rule, is_regex))

    def add_end_rule(self, rule, is_regex=False):
        self.end_rules.append((rule, is_regex))

    def extract(self, text, extract_if_no_start=False, extract_if_no_end=False):
        if len(self.start_rules) > 0 and not extract_if_no_start:
            start_index = None
        else:
            start_index = 0

        if len(self.end_rules) > 0 and not extract_if_no_end:
            end_index = None
        else:
            end_index = len(text)



        for rule, is_regex in self.start_rules:
            if is_regex:
                match = re.search(rule, text)
                if match is not None:
                    start_index = match.end()  # We want the index after the start rule
                    break  # If we've found a match, we can break
            else:
                idx = text.find(rule)
                if idx != -1:
                    start_index = idx + len(rule)  # We want the index after the start rule
                    break  # If we've found a match, we can break

        for rule, is_regex in self.end_rules:
            if is_regex:
                match = re.search(rule, text[start_index if start_index is not None else 0:])
                if match is not None:
                    end_index = (start_index if start_index is not None else 0) + match.start()  # We want the index before the end rule
                    break  # If we've found a match, we can break
            else:
                idx = text.find(rule, start_index if start_index is not None else 0)  # We search after the start index
                if idx != -1:
                    end_index = idx
                    break  # If we've found a match, we can break

        if start_index is None or end_index is None:
            return ''
        
        return text[start_index:end_index]





In [94]:

chrome_options = Options()
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
driver = webdriver.Chrome(options=chrome_options)

driver.get('https://www.linkedin.com/')

# Load cookies if they exist
try:
    cookies = pickle.load(open("linkedin_cookies.pkl", "rb"))
    for cookie in cookies:
        driver.add_cookie(cookie)
    driver.refresh()
except:
    print("No cookies found. Manual login required.")

# Check if login is needed
try: # not working
    element_present = EC.presence_of_element_located((By.ID, 'profile-nav-item')) # check for an element that is only present when logged in
    WebDriverWait(driver, 10).until(element_present)
except:
    # If not logged in
    input('Please login and press Enter to continue...')
    pickle.dump(driver.get_cookies(), open("linkedin_cookies.pkl", "wb")) # save cookies after login


# driver.quit()


Please login and press Enter to continue...


In [95]:
instructions = """
Extract the names: Please look for patterns indicating a person's name. Names are usually proper nouns and are generally the first information provided in each data point.

Find the job titles: Please identify each person's job title. This information follows the name and often contains phrases such as 'Data Scientist', 'Neuroscientist', 'Senior Associate Data Scientist', or a variant that includes the role and any additional qualifiers.

Identify the job location: This information typically follows the job title and is usually a city or metropolitan area, such as 'Pasadena, CA' or 'Los Angeles Metropolitan Area'.

Find the company names: The company names usually follow the job location. In case there is no current company mentioned, or the person is actively seeking a job, make a note of that by using placeholders like 'None (Job Seeking)' or 'None (No Company Mentioned)'.

Construct the dictionary: Organize all the extracted information into a dictionary format where the key represents the type of information (e.g., 'Name', 'Job Titles', 'Job Location', 'Company Name') and the value is a list of all the extracted information of that type in the order of appearance.

Formatting the instructions: Always make sure to start the code block with a comment '##BEGIN##', fill in the data accordingly, and end it with '##END##'. All columns of data should align with each other.

For example:


##BEGIN##

DF_TEMP = {
    "Name": ["Person 1", "Person 2", ...],
    "Job Titles": ["Job 1", "Job 2", ...],
    "Job Location": ["Location 1", "Location 2", ...],
    "Company Name": ["Company 1", "Company 2", ...]
}

##END##


Replace "Person 1", "Job 1", "Location 1", "Company 1", etc., with the actual extracted data.

Please make sure to follow these instructions to achieve consistent results.


here is the text:



"""

In [96]:

# def string_parcer_search_page(s):
#     pattern = r"(?<=search result pages)(.*?)(?=Page \d+ of \d+)"
#     match = re.search(pattern, s, re.DOTALL)
#     if match:
#         print("Match found!")
#         string_to_GPT = match.group().strip()
#         return string_to_GPT
#     else:
#         return None



In [79]:
# url_text = find_linkedin_info(driver, url_in)
# text_finder = Extractor()
# text_finder.add_start_rule('search result pages', False)
# text_finder.add_end_rule('Page \d+ of \d+', True)
# text_finder.add_end_rule("messaging overlay", False)
# out = text_finder.extract(url_text)
# print(out)



In [83]:
# url_in = 'https://www.linkedin.com/search/results/people/?keywords=data%20scientist&origin=CLUSTER_EXPANSION&sid=fRq'
# driver.get(url_in)

 



In [85]:
# # If finding by ID fails, try to find the element by aria-label
# next_button = WebDriverWait(driver, 10).until(
#     EC.presence_of_element_located((By.CSS_SELECTOR, "button[aria-label='Next']"))
# )
# next_button.click()




In [98]:

def click_next_button(driver):
    
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "button[aria-label='Next']"))
        )
        next_button.click()

    except Exception as e2:
        assert(False, f"Failed to find by aria-label, error: {str(e2)}")

        


<>:10: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:10: SyntaxWarning: assertion is always true, perhaps remove parentheses?
/var/folders/_t/ngsg61p93yg8mhy97ltx2n_80000gn/T/ipykernel_25275/991688616.py:10: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(False, f"Failed to find by aria-label, error: {str(e2)}")


In [100]:
url_in = 'https://www.linkedin.com/search/results/people/?keywords=data%20scientist&origin=CLUSTER_EXPANSION&sid=fRq'
driver.get(url_in)

time.sleep(10)
click_next_button(driver)
time.sleep(10)
click_next_button(driver)
time.sleep(10)




In [108]:
click_next_button(driver)

In [61]:
url_text = find_linkedin_info(driver)
text_finder = Extractor()
text_finder.add_start_rule('search result pages', False)
text_finder.add_end_rule('Page \d+ of \d+', True)
text_finder.add_end_rule("these results helpful", False)
text_finder.add_end_rule("messaging overlay", False)

gpt_text = text_finder.extract(url_text)
GPT_INPUT = instructions+gpt_text

In [62]:
# print(GPT_INPUT)

In [63]:


messages = [ {"role": "system", "content": "You are a intelligent assistant."} ]
messages.append({"role": "user", "content": GPT_INPUT})
chat = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages)
reply = chat.choices[0].message.content


In [64]:
print(reply)

##BEGIN##

DF_TEMP = {
    "Name": ["Abhinav R.", "Shijie (Selene) Xiang", "Jinze Xin", "Syamil Mohd Razak, PhD", "Justin Chen", "Malavika Ajith Nair", "Marshall R.", "Kevin Goldberg", "hayden thornton", "Lu L."],
    "Job Titles": ["Incoming Data Scientist at Chevron", "Data Scientist at eBay", "Data Scientist at Twitter", "Data Scientist | Petroleum Engineer", "Data Scientist at Snap Inc.", "Data Scientist", "Data Scientist | web3/DeFi", "Data Scientist at Meta", "Data Scientist at McKinsey & Company", "Data Scientist at Facebook"],
    "Job Location": ["Los Angeles, CA", "Los Angeles, CA", "Los Angeles Metropolitan Area", "Los Angeles Metropolitan Area", "Los Angeles Metropolitan Area", "United States", "United States", "Los Angeles, CA", "Los Angeles, CA", "Los Angeles, CA"],
    "Company Name": ["Chevron", "eBay", "Twitter", "Phillips 66", "Snap Inc.", "Meta", "Stealth Startup", "Meta", "McKinsey & Company", "Facebook"]
}

##END##


In [65]:
import pandas as pd

# Extracted information from the text
##BEGIN##

DF_TEMP = {
    "Name": ["Abhinav R.", "Jinze Xin", "Syamil Mohd Razak, PhD", "Shijie (Selene) Xiang", "Malavika Ajith Nair", "Marshall R.", "Mehrnaz Motamed", "Farimah Shirmohammadi, PhD", "Kevin Goldberg", "Lu L."],
    "Job Titles": ["Incoming Data Scientist at Chevron | USC Alum", "Data Scientist at Twitter", "Data Scientist | Petroleum Engineer", "Data Scientist at eBay", "Data Scientist", "Data Scientist | web3/DeFi", "Data Scientist - Machine Learning | Deep Learning | Statistics | R | SQL | Python", "Data Scientist", "Data Scientist at Meta", "Data Scientist at Facebook"],
    "Job Location": ["Los Angeles, CA", "Los Angeles Metropolitan Area", "Los Angeles Metropolitan Area", "Los Angeles, CA", "United States", "United States", "Pasadena, CA", "Los Angeles, CA", "Los Angeles, CA", "Los Angeles, CA"],
    "Company Name": ["Chevron", "Twitter", "Phillips 66", "eBay", "Meta", "Stealth Startup", "Endura Technologies", "Edison International", "Meta", "Facebook"]
}


# Create a DataFrame
DF_TEMP = pd.DataFrame(DF_TEMP)



In [66]:
DF_TEMP

,Name,Job Titles,Job Location,Company Name
0,Abhinav R.,Incoming Data Scientist at Chevron | USC Alum,"Los Angeles, CA",Chevron
1,Jinze Xin,Data Scientist at Twitter,Los Angeles Metropolitan Area,Twitter
2,"Syamil Mohd Razak, PhD",Data Scientist | Petroleum Engineer,Los Angeles Metropolitan Area,Phillips 66
3,Shijie (Selene) Xiang,Data Scientist at eBay,"Los Angeles, CA",eBay
4,Malavika Ajith Nair,Data Scientist,United States,Meta
5,Marshall R.,Data Scientist | web3/DeFi,United States,Stealth Startup
6,Mehrnaz Motamed,Data Scientist - Machine Learning | Deep Learn...,"Pasadena, CA",Endura Technologies
7,"Farimah Shirmohammadi, PhD",Data Scientist,"Los Angeles, CA",Edison International
8,Kevin Goldberg,Data Scientist at Meta,"Los Angeles, CA",Meta
9,Lu L.,Data Scientist at Facebook,"Los Angeles, CA",Facebook
